In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import json
from urllib.parse import urljoin

# Scraping

In [ ]:
# Charger les villes depuis le JSON de référence
with open("../data/config/cities_best.json", "r", encoding="utf-8") as f:
    cities = json.load(f)

print("Villes à scraper :", cities)

# Configuration Selenium et création du driver
options = Options()
options.add_argument("--headless=new")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/123.0.0.0 Safari/537.36"
)

driver = webdriver.Chrome(options=options)

hotels = []

try:
    # Boucle sur les villes
    for city in cities:
        print(f"\nScraping Booking pour : {city}")

        search_url = (
            "https://www.booking.com/searchresults.fr.html"
            f"?ss={city}&lang=fr&dest_type=city"
        )

        driver.get(search_url)
        time.sleep(5)

        soup = BeautifulSoup(driver.page_source, "html.parser")

        cards = soup.select('[data-testid="property-card-container"]')
        print("Nombre d'hôtels trouvés sur la page de résultats :", len(cards))

        # Boucle sur les hôtels de la ville
        for card in cards:
            name_el = card.select_one('[data-testid="title"]')
            link_el = card.select_one('a[data-testid="title-link"]')
            score_el = card.select_one('[data-testid="review-score"]')
            addr_search_el = card.select_one('[data-testid="address"]')

            nom = name_el.get_text(strip=True) if name_el else None
            url_hotel = (
                urljoin(search_url, link_el["href"])
                if link_el is not None and link_el.has_attr("href")
                else None
            )
            note = score_el.get_text(" ", strip=True) if score_el else None
            adresse_search = (
                addr_search_el.get_text(" ", strip=True) if addr_search_el else None
            )

            description = None
            adresse_full = None
            lat = None
            lng = None

            if url_hotel:
                driver.get(url_hotel)
                time.sleep(4)

                soup_hotel = BeautifulSoup(driver.page_source, "html.parser")

                # Description principale
                desc_tag = soup_hotel.select_one('p[data-testid="property-description"]')
                if desc_tag:
                    description = desc_tag.get_text(" ", strip=True)

                # Adresse complète
                addr_tag = soup_hotel.select_one(
                    'div.b99b6ef58f.cb4b7a25d9.b06461926f'
                )
                if addr_tag:
                    adresse_full = addr_tag.get_text(" ", strip=True)

                # Coordonnées GPS
                map_div = soup_hotel.select_one('div[data-atlas-latlng]')
                if map_div:
                    coords = map_div.get("data-atlas-latlng")
                    if coords and "," in coords:
                        lat, lng = [c.strip() for c in coords.split(",", 1)]

            hotels.append(
                {
                    "ville": city,
                    "nom": nom,
                    "note": note,
                    "adresse_search": adresse_search,
                    "adresse": adresse_full,
                    "description": description,
                    "lat": lat,
                    "lng": lng,
                    "url": url_hotel,
                }
            )

            time.sleep(1)

finally:
    # Fin Selenium
    driver.quit()

print(f"\nScraping terminé. Total hôtels collectés : {len(hotels)}")

Villes à scraper : ['Collioure', 'Aigues Mortes', 'Bayeux', 'Marseille', 'Cassis']

Scraping Booking pour : Collioure
Nombre d'hôtels trouvés sur la page de résultats : 25

Scraping Booking pour : Aigues Mortes
Nombre d'hôtels trouvés sur la page de résultats : 25

Scraping Booking pour : Bayeux
Nombre d'hôtels trouvés sur la page de résultats : 25

Scraping Booking pour : Marseille
Nombre d'hôtels trouvés sur la page de résultats : 25

Scraping Booking pour : Cassis
Nombre d'hôtels trouvés sur la page de résultats : 25

Scraping terminé. Total hôtels collectés : 125


# Dataframe

In [7]:
df_hotels = pd.DataFrame(hotels)

In [8]:
df_hotels

,ville,nom,note,adresse_search,adresse,description,lat,lng,url
0,Collioure,Les Roches Brunes,"Avec une note de 9,2 9,2 Fabuleux 761 expérien...",Collioure,"Route de Port-Vendres, 66190 Collioure, France...",L’établissement Les Roches Brunes se trouve à ...,42.52534777273077,3.0903244632682725,https://www.booking.com/hotel/fr/les-roches-br...
1,Collioure,Maison Nova - Boutique Hôtel & Spa,"Avec une note de 9,2 9,2 Fabuleux 530 expérien...",Collioure,"33 Route du Pla de las Fourques, 66190 Colliou...",L’établissement Maison Nova - Boutique Hôtel &...,42.5288828,3.0793938,https://www.booking.com/hotel/fr/boutique-mais...
2,Collioure,Hôtel La Casa Pairal,"Avec une note de 9,1 9,1 Fabuleux 948 expérien...",Collioure,"Impasse Des Palmiers, 66190 Collioure, France ...",Situé à 250 mètres de la plage et du port de C...,42.5261673320343,3.082355707883835,https://www.booking.com/hotel/fr/hotelcasapair...
3,Collioure,Hôtel & Spa Le Madeloc - Collioure Centre,"Avec une note de 9,0 9,0 Fabuleux 914 expérien...",Collioure,"24 Rue Romain Rolland, 66190 Collioure, France...",Le Madeloc Hôtel & Spa se trouve à 500 mètres ...,42.5287646672607,3.078654259443283,https://www.booking.com/hotel/fr/madeloc.fr.ht...
4,Collioure,La Frégate,"Avec une note de 8,2 8,2 Très bien 1 615 expér...",Collioure,"24 Boulevard Camille Pelletan, 66190 Collioure...","Situé à Collioure, l’établissement La Frégate ...",42.52638625163285,3.083256594836712,https://www.booking.com/hotel/fr/la-fra-c-gate...
...,...,...,...,...,...,...,...,...,...
120,Cassis,Entre Ciel et Mer Appartement Cassis,"Avec une note de 9,2 9,2 Fabuleux 19 expérienc...",Cassis,"1 Avenue de l'Amiral Ganteaume, 13260 Cassis, ...","Situé à Cassis, l’hébergement Entre Ciel et Me...",43.21440700000001,5.534905,https://www.booking.com/hotel/fr/spacious-apar...
121,Cassis,Best Western Plus Hôtel la Rade,"Avec une note de 8,7 8,7 Superbe 946 expérienc...",Cassis,"1, avenue des Dardanelles, 13260 Cassis, Franc...",Le Best Western Plus Hôtel la Rade est situé à...,43.21522996680949,5.533696934580803,https://www.booking.com/hotel/fr/la-rade-cassi...
122,Cassis,LE SEPT charmant studio aux portes des calanques,"Avec une note de 9,8 9,8 Exceptionnel 100 expé...",Cassis,"7 Avenue des Carriers, 13260 Cassis, France Ex...","Situé à Cassis et offrant une vue sur la mer, ...",43.2175997,5.53235,https://www.booking.com/hotel/fr/le-sept-charm...
123,Cassis,Au son des Cigales,"Avec une note de 9,1 9,1 Fabuleux 62 expérienc...",Cassis,"étage 2 16 Avenue du Picouveau, 13260 Cassis, ...",L’hébergement 1 étoile Au son des Cigales se s...,43.215086781027,5.530949017791,https://www.booking.com/hotel/fr/au-son-des-ci...


In [9]:
df_hotels.shape

(125, 9)

In [10]:
# Processing de la colonne note
df_hotels["note_cleaned"] = df_hotels["note"].str.extract(r"(\d,\d)")
df_hotels["note_cleaned"].str.replace(",", ".").astype(float)
df_hotels["rating"] = df_hotels["note"].str.split().str[6]

In [11]:
df_hotels = df_hotels.drop("note", axis=1)

In [12]:
df_hotels = df_hotels.rename(columns={"note_cleaned": "note"})
df_hotels

,ville,nom,adresse_search,adresse,description,lat,lng,url,note,rating
0,Collioure,Les Roches Brunes,Collioure,"Route de Port-Vendres, 66190 Collioure, France...",L’établissement Les Roches Brunes se trouve à ...,42.52534777273077,3.0903244632682725,https://www.booking.com/hotel/fr/les-roches-br...,"9,2",Fabuleux
1,Collioure,Maison Nova - Boutique Hôtel & Spa,Collioure,"33 Route du Pla de las Fourques, 66190 Colliou...",L’établissement Maison Nova - Boutique Hôtel &...,42.5288828,3.0793938,https://www.booking.com/hotel/fr/boutique-mais...,"9,2",Fabuleux
2,Collioure,Hôtel La Casa Pairal,Collioure,"Impasse Des Palmiers, 66190 Collioure, France ...",Situé à 250 mètres de la plage et du port de C...,42.5261673320343,3.082355707883835,https://www.booking.com/hotel/fr/hotelcasapair...,"9,1",Fabuleux
3,Collioure,Hôtel & Spa Le Madeloc - Collioure Centre,Collioure,"24 Rue Romain Rolland, 66190 Collioure, France...",Le Madeloc Hôtel & Spa se trouve à 500 mètres ...,42.5287646672607,3.078654259443283,https://www.booking.com/hotel/fr/madeloc.fr.ht...,"9,0",Fabuleux
4,Collioure,La Frégate,Collioure,"24 Boulevard Camille Pelletan, 66190 Collioure...","Situé à Collioure, l’établissement La Frégate ...",42.52638625163285,3.083256594836712,https://www.booking.com/hotel/fr/la-fra-c-gate...,"8,2",Très
...,...,...,...,...,...,...,...,...,...,...
120,Cassis,Entre Ciel et Mer Appartement Cassis,Cassis,"1 Avenue de l'Amiral Ganteaume, 13260 Cassis, ...","Situé à Cassis, l’hébergement Entre Ciel et Me...",43.21440700000001,5.534905,https://www.booking.com/hotel/fr/spacious-apar...,"9,2",Fabuleux
121,Cassis,Best Western Plus Hôtel la Rade,Cassis,"1, avenue des Dardanelles, 13260 Cassis, Franc...",Le Best Western Plus Hôtel la Rade est situé à...,43.21522996680949,5.533696934580803,https://www.booking.com/hotel/fr/la-rade-cassi...,"8,7",Superbe
122,Cassis,LE SEPT charmant studio aux portes des calanques,Cassis,"7 Avenue des Carriers, 13260 Cassis, France Ex...","Situé à Cassis et offrant une vue sur la mer, ...",43.2175997,5.53235,https://www.booking.com/hotel/fr/le-sept-charm...,"9,8",Exceptionnel
123,Cassis,Au son des Cigales,Cassis,"étage 2 16 Avenue du Picouveau, 13260 Cassis, ...",L’hébergement 1 étoile Au son des Cigales se s...,43.215086781027,5.530949017791,https://www.booking.com/hotel/fr/au-son-des-ci...,"9,1",Fabuleux


In [14]:
# Export du dataframe
df_hotels.to_csv("../data/outputs/booking_hotels_top5_cities.csv")